### Program 1
- In this assignment you will write a series of programs to solve sliding-tile puzzles using various uninformed and informed (heuristic) methods.
- **Heuristics**, or "rules of thumb," are problem-solving methods that are based on practical experience and knowledge. They allow you to use a "quick fix" to solve a minor problem or to narrow down options.
#### Part 1 – Reading and Validating Sliding-Puzzle Problems
- Read a sliding-tile puzzle problem using a JSON parser. 
- Check that the sliding-tile puzzle problem is valid. Specifically, there must be fields named *n*, *start*, and *goal*. The n field must be a positive integer greater than 1. The start and goal fields must be 𝑛 x 𝑛 matrices containing the integers 0 (for the empty space) to 𝑛2 − 1

In [35]:
import pandas as pd

df1 = pd.read_json('datasets/1-move.json')
df2 = pd.read_json('datasets/2-moves.json')
df2


,n,start,goal
0,3,"[3, 1, 2]","[0, 1, 2]"
1,3,"[4, 0, 5]","[3, 4, 5]"
2,3,"[6, 7, 8]","[6, 7, 8]"


In [41]:
import string

def validation(df: pd.DataFrame, filename: string):
    # check if n, start and goal are defined
    if(df.columns.__contains__('n') and df.columns.__contains__('start') and df.columns.__contains__('goal')):
        # check the value of n
        if(df['n'].unique()):
            nvalue = df['n'].unique()[0]
            # if n is greater than or equal to 1, proceed
            startLength = len(df['start'])
            goalLength = len(df['goal'])
            # if the start and goal columns are of length n, proceed
            if (nvalue >= 1 and startLength == nvalue and goalLength == nvalue):
                print("'n' is greater than or equal to one. 'n' is", nvalue)
                # check the start and goal columns
                # if they are of size n AND contain a zero, proceed 
                for row,row2 in zip(df['start'],df['goal']):
                    if len(row) == nvalue and len(row2) == nvalue: 
                        if 0 in row or 0 in row2:
                            print('0 found in', row,row2)
            print(f'{filename} is valid' )
        else:
            print(f'{filename} is not valid. n is not greather than or equal to one.') 
    else:
        print(f"{filename} is not valid.JSON file does not contain fields 'n', 'start', or 'goal'")

validation(df1, '1-move.json')
validation(df2, "2-moves.json")

'n' is greater than or equal to one. 'n' is 3
0 found in [3, 1, 2] [0, 1, 2]
0 found in [0, 4, 5] [3, 4, 5]
1-move.json is valid
'n' is greater than or equal to one. 'n' is 3
0 found in [3, 1, 2] [0, 1, 2]
0 found in [4, 0, 5] [3, 4, 5]
2-moves.json is valid


#### Part 2 – Sliding-Tile Puzzle Rules
- Given a sliding-tile puzzle state, you must be able to determine the rules that are applicable to that state that can be used to generate its successor states.
- A rule has three parts:
    - *name* – a simple name for the rule (e.g., up, left, down, right)
    - *precondition function* – a Boolean function that accepts a state and returns true if the rule is applicable to state
    - *action function* – a function that accepts a state and returns the successor state obtained by applying the rule